##### Загрузка библиотек

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

##### Загрузка данных

In [2]:
data = pd.read_excel('data/data_ford_price.xlsx')

##### Предобработка данных

In [3]:
data = data[['price','year', 'cylinders', 'odometer', 'lat', 'long', 'weather']]
data.dropna(inplace = True)

##### Разделим набор данных на матрицу наблюдений x и вектор правильных ответов y:

In [4]:
x = data.drop(columns='price')
y = data['price']

##### Разделим выборку в соотношении 70/30 (test_size=0.3), в качестве значения параметра random_state по возьмём число 40.

In [5]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=40)

##### Выделим три наиболее значимых признака с помощью метода рекурсивного исключения признаков (RFE):

In [6]:
from sklearn.feature_selection import RFE

estimator = LinearRegression()
selector = RFE(estimator, n_features_to_select=3, step=1)
selector = selector.fit(X_train, y_train)
 
selector.get_feature_names_out()

array(['year', 'cylinders', 'lat'], dtype=object)

##### RFE определил следующие три наиболее значимых признака: year, cylinders, lat.

##### Обучим модель линейной регрессии на найденных трёх наиболее значимых признаках методом RFE и оценим ее качество с помощью метрики MAE:

In [7]:
data_rfe = data[['year', 'cylinders', 'lat']]
x = data_rfe
y = data['price']

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=40)
 
model = LinearRegression()
model.fit(X_train, y_train)
y_predicted_rfe = model.predict(X_test)

mae = mean_absolute_error(y_test, y_predicted_rfe)
print('MAE: %.2f' % mae)

MAE: 5096.57


##### Выделим три наиболее значимых признака с помощью метода выбора k лучших переменных (SelectKBest):

In [8]:
from sklearn.feature_selection import SelectKBest, f_regression

selector = SelectKBest(f_regression, k=3)
selector.fit(X_train, y_train)
 
selector.get_feature_names_out()

array(['year', 'cylinders', 'lat'], dtype=object)

##### SelectKBest определил следующие три наиболее значимых признака: year, cylinders, odometer.

##### Обучим модель линейной регрессии на найденных трёх наиболее значимых признаках методом SelectKBest и оценим ее качество с помощью метрики MAE:

In [9]:
data_skb = data[['year', 'cylinders', 'odometer']]
x = data_skb
y = data['price']

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=40)
 
model = LinearRegression()
model.fit(X_train, y_train)
y_predicted_skb = model.predict(X_test)

mae = mean_absolute_error(y_test, y_predicted_skb)
print('MAE: %.2f' % mae)

MAE: 4708.95


##### Вывод: средняя абсолютная ошибка (MAE) при обучении модели на признаках, определенных методом RFE равна 5096.57, а на признаках, определенных методом SelectKBest равна 4708.95. Из этого следует, что метод выбора k лучших переменных: SelectKBest показал наилучший результат на тестовой выборке.